In [1]:
# Install Hugging Face Transformers library
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:

# Import necessary libraries
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Load GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
# Make sure the tokenizer pad token is set (GPT-2 does not use a padding token by default)
tokenizer.pad_token = tokenizer.eos_token

# Replace 'your_dataset_path' with the path to your text data
dataset = load_dataset("text", data_files={"train": "/content/dataset.txt"})

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Prepare the dataset for PyTorch
tokenized_dataset = tokenized_dataset["train"].map(lambda examples: {'labels': examples['input_ids']}, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    evaluation_strategy="no",
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    report_to="none"
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

# Train the model
trainer.train()

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=15, training_loss=8.178179931640624, metrics={'train_runtime': 206.2135, 'train_samples_per_second': 0.291, 'train_steps_per_second': 0.073, 'total_flos': 3919380480000.0, 'train_loss': 8.178179931640624, 'epoch': 3.0})

In [6]:

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")

# Load the fine-tuned model for text generation
from transformers import pipeline

fine_tuned_model = GPT2LMHeadModel.from_pretrained("./fine_tuned_gpt2")
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained("./fine_tuned_gpt2")

text_generator = pipeline("text-generation", model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)

In [8]:
# Generate text dynamically based on user input
user_prompt = input("Enter your prompt: ")  # Get prompt from the user
generated_text = text_generator(user_prompt, max_length=50, num_return_sequences=1)

# Print the generated text
print("Generated Text:", generated_text[0]['generated_text'])


Enter your prompt: Once upon a time
Generated Text: Once upon a time, many other cities along this coast of Italy and Italy, across western and southern Italy and the Mediterranean Sea, had populations of about 2200 to 3000, or approximately 2- and 2-5 times as many as the present-


In [9]:
# Generate text dynamically based on user input
user_prompt = input("Enter your prompt: ")  # Get prompt from the user
generated_text = text_generator(user_prompt, max_length=50, num_return_sequences=1)

# Print the generated text
print("Generated Text:", generated_text[0]['generated_text'])

Enter your prompt: In the heart of the ancient forest
Generated Text: In the heart of the ancient forest area, an enormous, serpentine-looking thing rises from the snow and comes to rest next to us. An elf girl stands on the left and a lizard-like creature appears from the snow and sits in front
